In [67]:
import pandas as pd
import numpy as np

In [68]:
file_to_load="purchase_data.csv"

In [69]:
purchase_data = pd.read_csv("purchase_data.csv")

In [92]:
# Player Count

total_players = purchase_data.SN.nunique() #players are in data set multiple times
total_players_series = pd.Series({"Total Players": total_players})
total_players_series

Total Players    576
dtype: int64

In [116]:
# Purchasing Analysis (Total)

num_unique_items = purchase_data['Item Name'].nunique()
avg_purchase_price = purchase_data.Price.mean()
num_purchases = purchase_data['Purchase ID'].count()
total_revenue = purchase_data.Price.sum()


In [123]:
purchasing_analysis_df = pd.Series({
    "Number of Unique Items": int(num_unique_items), 
    "Average Price": round(avg_purchase_price, 2), 
    "Number of Purchases:": int(num_purchases),
    "Total Revenue:": round(total_revenue,2)})
purchasing_analysis_df

Number of Unique Items     179.00
Average Price                3.05
Number of Purchases:       780.00
Total Revenue:            2379.77
dtype: float64

In [37]:
# Gender Demographics

#Percentage and count of players
gender_count = purchase_data.groupby('Gender').SN.nunique() #same SN in file multiple times due to multiple purchases
gender_perc = round(gender_count / total_players * 100,2)

In [38]:
gender_demographics_df = pd.DataFrame({"Total Count":gender_count, "Percentage of Players":gender_perc})
gender_demographics_df

,Total Count,Percentage of Players
Gender,,
Female,81,14.06
Male,484,84.03
Other / Non-Disclosed,11,1.91


In [19]:
# Purchasing Analysis (Gender)

purchase_count_per_gender = purchase_data.groupby("Gender")["Purchase ID"].count()
num_persons_per_gender = purchase_data.groupby("Gender").SN.nunique()
avg_purchase_price_per_gender = purchase_data.groupby("Gender").Price.mean()
total_purchase_value_per_gender = purchase_data.groupby("Gender").Price.sum()
avg_purchase_per_person_per_gender = total_purchase_value_per_gender / num_persons_per_gender


In [122]:
purchasing_analysis_df = pd.DataFrame({
    "Purchase Count": purchase_count_per_gender, 
    "Avg Purchase Price": round(avg_purchase_price_per_gender,2),
    "Total Purchase Value": total_purchase_value_per_gender,
    "Avg Total Purchase per Person": round(avg_purchase_per_person_per_gender,2)})
purchasing_analysis_df

,Purchase Count,Avg Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Gender,,,,
Female,113,3.20,361.94,4.47
Male,652,3.02,1967.64,4.07
Other / Non-Disclosed,15,3.35,50.19,4.56


In [ ]:
# Age Demographics

max_age = purchase_data.Age.max()

In [21]:
#Deterine number of bins needed. Divide by 5-year groupings. Subtract 9 for first group 0 - 9 years old.
num_bins = (max_age - 9) / 5

#If division by 5 returns a remainder, need an additional bin for that 0.xx person.
if num_bins % 5 != 0:
    num_bins += 1

#Create bins_list containing upper ages of age ranges.
#Create labels_lists containing age ranges.
bins_list = [0, 9] #accounting for first range < 10 years old
labels_list = ["<10"]
upper_bin_age = 9
for i in range(int(num_bins)):
    upper_bin_age += 5
    bins_list.append(upper_bin_age)
    labels_list.append(str(upper_bin_age - 4) + "-" + str(upper_bin_age))

In [ ]:
#create new column 'age_bin'
purchase_data['age_bin'] = pd.cut(purchase_data['Age'], bins=bins_list, labels=labels_list)

In [112]:
#Total count and Percentage of Players per age group

player_count_by_age_group = purchase_data.groupby('age_bin').SN.nunique()
player_percentage_by_age_group = round(player_count_by_age_group / total_players * 100, 2)

age_demographics_df = pd.DataFrame({
    "Total Count": player_count_by_age_group,
    "Percentage": player_percentage_by_age_group
})
age_demographics_df

,Total Count,Percentage
age_bin,,
<10,17,2.95
10-14,22,3.82
15-19,107,18.58
20-24,258,44.79
25-29,77,13.37
30-34,52,9.03
35-39,31,5.38
40-44,11,1.91
45-49,1,0.17


In [108]:
# Purchasing Analysis (Age)

purchase_count_by_age_bin = purchase_data.groupby('age_bin').Price.count()
num_persons_in_age_bin = purchase_data.groupby('age_bin').SN.nunique()
avg_purchase_price_by_age_bin = purchase_data.groupby('age_bin').Price.mean()
total_purchases_by_age_bin = purchase_data.groupby('age_bin').Price.sum()
avg_purchase_per_person_by_age_bin = round(total_purchases_by_age_bin / num_persons_in_age_bin, 2)

In [109]:
age_demographics_df = pd.DataFrame ({
    "Purchase Count": purchase_count_by_age_bin,
    "Average Purchase Price": avg_purchase_price_by_age_bin,
    "Total Purchase Value": total_purchases_by_age_bin,
    "Average Purchase Total Per Person": avg_purchase_per_person_by_age_bin
})
age_demographics_df

,Purchase Count,Average Purchase Price,Total Purchase Value,Average Purchase Total Per Person
age_bin,,,,
<10,23,3.353478,77.13,4.54
10-14,28,2.956429,82.78,3.76
15-19,136,3.035956,412.89,3.86
20-24,365,3.052219,1114.06,4.32
25-29,101,2.900990,293.00,3.81
30-34,73,2.931507,214.00,4.12
35-39,41,3.601707,147.67,4.76
40-44,12,3.045000,36.54,3.32
45-49,1,1.700000,1.70,1.70


In [103]:
# Top Spenders

top_spenders_df = purchase_data.loc[:, ["SN", "Item ID", "Price"]]

ts_purchase_count = top_spenders_df.groupby("SN")["Item ID"].count()
ts_purchase_value = top_spenders_df.groupby("SN").Price.sum()
ts_avg_purchase_price = ts_purchase_value / ts_purchase_count

ts_results_df = pd.DataFrame ({
    "Purchase Count": ts_purchase_count,
    "Avg Purchase Price": ts_avg_purchase_price,
    "Total Purchase Value": ts_purchase_value
})
top_5_spenders = ts_results_df.sort_values("Total Purchase Value", ascending=False).head(5)
top_5_spenders

,Purchase Count,Avg Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,3.792000,18.96
Idastidru52,4,3.862500,15.45
Chamjask73,3,4.610000,13.83
Iral74,4,3.405000,13.62
Iskadarya95,3,4.366667,13.10


In [107]:
# Most Popular Items

df = purchase_data.loc[:,['Item ID', 'Item Name', 'Price']]

purchase_count = df.groupby(['Item ID', 'Item Name'])['Item ID'].count()
purchase_value = df.groupby(['Item ID', 'Item Name']).Price.sum()
item_price = round(purchase_value / purchase_count, 2)

calcs_df = pd.DataFrame ({
    "Purchase Count": purchase_count,
    "Item Price": item_price,
    "Purchase Value": purchase_value
})

most_popular_df = calcs_df.sort_values("Purchase Count", ascending=False).head(5)
most_popular_df

,,Purchase Count,Item Price,Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,4.23,50.76
145,Fiery Glass Crusader,9,4.58,41.22
108,"Extraction, Quickblade Of Trembling Hands",9,3.53,31.77
82,Nirvana,9,4.90,44.10
19,"Pursuit, Cudgel of Necromancy",8,1.02,8.16


In [104]:
# Most Profitable items

most_profitable_df =  calcs_df.sort_values("Purchase Value", ascending=False).head(5)
most_profitable_df


,,Purchase Count,Item Price,Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,4.23,50.76
82,Nirvana,9,4.90,44.10
145,Fiery Glass Crusader,9,4.58,41.22
92,Final Critic,8,4.88,39.04
103,Singed Scalpel,8,4.35,34.80
